In [1]:
from mistralai import Mistral
import yaml
import json
import requests 
from tqdm.auto import notebook_tqdm
from elasticsearch import Elasticsearch

# from elasticsearch import Elasticsearch

In [2]:
api_key = "ollama"
model = "mistral"

In [3]:
client = Mistral(server_url="http://127.0.0.1:11434", api_key=api_key)

In [ ]:
# chat_response = client.chat.complete(
#     model=model,
#     messages=[{"role":"user", "content":"What is the best French cheese?"}]
# )

In [4]:
# print(chat_response)

Aqui puedo modificar la URL a la que apunta el codigo, en lugar de a la de mistral api, a la del puerto desde el que se puede acceder a ollama. No necesitaria una key porque mi puerto esta abierto sino que tendria que apuntar a la aplicacion ollama para que se ejecute. luego podria pedirle un modelo de ollama, en este caso podria usar mistral 7b en lugar de mistral small latest.


1. hay que hacer ollama serve para que se habra la url 
2. luego hay que hacer ollama pull para que descargue el modelo
3. a partir de ahi se pude usar

In [5]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [6]:
es_client = Elasticsearch(
    hosts=[{'host': 'localhost', 'port': 9200, 'scheme': 'http'}],
    timeout=30,
    max_retries=10,
    retry_on_timeout=True
)

# Prueba la conexión
if es_client.ping():
    print("Conectado a Elasticsearch!")
else:
    print("No se pudo conectar a Elasticsearch.")

Conectado a Elasticsearch!


/tmp/ipykernel_36813/395609063.py:1: DeprecationWarning: The 'timeout' parameter is deprecated in favor of 'request_timeout'
  es_client = Elasticsearch(


In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-ollama2"

es_client.indices.create(index=index_name, body=index_settings)

for doc in notebook_tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [9]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"], #the power gives more importance to that part
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp" #we add a filter for one of the keywords on the documents 
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [10]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
        Use only the facts from the CONTEXT when answering the QUESTION.

        QUESTION: {question}

        CONTEXT: 
        {context}
        """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [11]:
def llm(prompt):
    response = client.chat.complete(
        model='mistral',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [12]:
query = 'how do I run kafka?'

def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [14]:
rag(query)

" To run Kafka, follow these steps based on the provided context:\n\n1. Ensure that you have the necessary dependencies to run the code, which is 'dlt[duckdb]'. You can install it using the command `!pip install dlt[duckdb]`.\n\n2. If you are working with Python and Kafka, create a virtual environment and install the required packages in that environment. To create a virtual env:\n   - On MacOS, Linux, or Windows (except for Windows, the path is slightly different): `python -m venv env` then `source env/bin/activate`\n   - For Windows: `python -m venv env` followed by `env\\Scripts\\activate`.\n\n3. Install Kafka dependencies in the virtual environment using pip:\n   - Run `pip install -r ../requirements.txt`. This command assumes that you have a 'requirements.txt' file with all the necessary packages for your project.\n\n4. Once you have created and activated the virtual environment, navigate to your project directory containing the Kafka producer or consumer files. Run the Java Kafka